# **DEPENDENCIES**  

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10

from tqdm import tqdm

# **DATA**

CIFAR-10 dataset: https://www.cs.toronto.edu/~kriz/cifar.html

In [ ]:
# Cifar10 dataset
Xtr = CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
Xte = CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

# Data loader
batch_size = 128
train_loader = DataLoader(Xtr, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(Xte, batch_size=batch_size, shuffle=False)

## **Q1**

In [9]:
config_space = {
    'kernel_size': [3, 5],
    'stride': [1, 2],
    'activation': ["Relu", "Sigmoid", "Gelu"],
    'pool_type': ["Max", "Avg"],
    'batch_norm': [True, False],
    'num_filter_1': [32, 64],
    'num_filter_2': [64, 128],
    'num_filter_3': [128, 256],
    'num_fc': [256, 512],
    'dropout': [0.2, 0.5],
}

activs = {
    'Relu': nn.ReLU(),
    'Sigmoid': nn.Sigmoid(),
    'Gelu': nn.GELU(),
}

pools = {
    'Max': nn.MaxPool2d(2),
    'Avg': nn.AvgPool2d(2),
}

Example:

```python
config = {
    'kernel_size': 3,
    'stride': 1,
    'activation': "Relu",
    'pool_type': "Max",
    'batch_norm': True,
    'num_filter_1': 32,
    'num_filter_2': 64,
    'num_filter_3': 128,
    'num_fc': 256,
    'dropout': 0.2,
}
```

In [10]:
class ConvNet(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.conv1 = nn.Conv2d(
            3,
            config["num_filter_1"],
            config["kernel_size"],
            config["stride"],
            padding=1,
        )
        self.conv2 = nn.Conv2d(
            config["num_filter_1"],
            config["num_filter_2"],
            config["kernel_size"],
            config["stride"],
            padding=1,
        )
        self.conv3 = nn.Conv2d(
            config["num_filter_2"],
            config["num_filter_3"],
            config["kernel_size"],
            config["stride"],
            padding=1,
        )
        self.pool = pools[config["pool_type"]]
        self.activation = activs[config["activation"]]
        self.batch_norm = config["batch_norm"]
        self.dropout = nn.Dropout(config["dropout"])
        self.fc1 = nn.Linear(config["num_filter_3"] * 4 * 4, config["num_fc"])
        self.fc2 = nn.Linear(config["num_fc"], 10)
        self.bn1 = nn.BatchNorm2d(config["num_filter_1"])
        self.bn2 = nn.BatchNorm2d(config["num_filter_2"])
        self.bn3 = nn.BatchNorm2d(config["num_filter_3"])
        self.bn4 = nn.BatchNorm1d(config["num_fc"])

    def forward(self, x):
        x = self.conv1(x)
        if self.batch_norm:
            x = self.bn1(x)
        x = self.activation(x)
        x = self.pool(x)
        x = self.conv2(x)
        if self.batch_norm:
            x = self.bn2(x)
        x = self.activation(x)
        x = self.pool(x)
        x = self.conv3(x)
        if self.batch_norm:
            x = self.bn3(x)
        x = self.activation(x)
        x = self.pool(x)
        x = x.view(-1, self.config["num_filter_3"] * 4 * 4)
        x = self.dropout(x)
        x = self.fc1(x)
        if self.batch_norm:
            x = self.bn4(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

#### **Training**


In [8]:
# Generate random config
def generate_random_config():
    config = {}
    for key in config_space.keys():
        config[key] = np.random.choice(config_space[key])
    return config

# Trainer
def train(config, train_loader, test_loader, num_epochs=10):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = ConvNet(config).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    for epoch in range(num_epochs):
        # Progress bar with prefix "Epoch: "
        pbar = tqdm(train_loader, desc=f"Epoch: {epoch+1}")
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            # pbar update live with postfix metrics
            pbar.set_postfix(
                loss=running_loss / (i + 1),
                acc=correct / total,
            )
        pbar.close()
        train_loss.append(running_loss / len(train_loader))
        train_acc.append(correct / total)
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for i, (images, labels) in enumerate(test_loader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        test_loss.append(running_loss / len(test_loader))
        test_acc.append(correct / total)
    return train_loss, test_loss, train_acc, test_acc

In [ ]:
results = {}

# Generate 32 random configs and add results to dictionary along with config
for i in tqdm(range(32)):
    config = generate_random_config()
    train_loss, test_loss, train_acc, test_acc = train(
        config, train_loader, test_loader, num_epochs=10
    )
    results[f"random_{i}"] = {
        "config": config,
        "train_loss": train_loss,
        "test_loss": test_loss,
        "train_acc": train_acc,
        "test_acc": test_acc,
    }

# Dataframe for config and metrics
results_df = pd.DataFrame(
    columns=["name", "config", "train_loss", "test_loss", "train_acc", "test_acc"]
)

# Add results to dataframe
for key in results.keys():
    results_df = results_df.append(
        {
            "name": key,
            "config": results[key]["config"],
            "train_loss": results[key]["train_loss"],
            "test_loss": results[key]["test_loss"],
            "train_acc": results[key]["train_acc"],
            "test_acc": results[key]["test_acc"],
        },
        ignore_index=True,
    )

# Show dataframe
print(results_df)

In [ ]:
# Best config
best_config = results_df.iloc[results_df["test_acc"].argmax()]["config"]
print(best_config)